# 네이버 검색 API - 월간 조회수

In [24]:
# 네이버 검색 API - 월간 조회수
import pandas as pd
import time
import requests

import signaturehelper
import urllib.request

def get_header(method, uri, api_key, secret_key, customer_id):
    timestamp = str(round(time.time() * 1000))
    signature = signaturehelper.Signature.generate(timestamp, method, uri, SECRET_KEY)
    return {'Content-type': 'application/json; charset=UTF-8', 'X-Timestamp': timestamp,
            'X-API-KEY': API_KEY, 'X-Customer': str(CUSTOMER_ID), 'X-Signature': signature}

BASE_URL = "https://api.searchad.naver.com"
API_KEY = "01000000009a28e817649a0e015c806c61bfe76fb2861d89f23615f6d57d11beed6e1add92"
SECRET_KEY = "AQAAAACaKOgXZJoOAVyAbGG/52+yCo+9cVSAUSUTKSAQeDlUhQ=="
CUSTOMER_ID = 3189735

uri = '/keywordstool'
method = 'GET'

keyword = "다음"

params={}
params['hintKeywords']=keyword
params['showDetail']='1'

response = requests.get(BASE_URL + uri, params=params, 
                        headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))
df2 = pd.DataFrame(response.json()["keywordList"])
print(df2)

  relKeyword  monthlyPcQcCnt  monthlyMobileQcCnt  monthlyAvePcClkCnt  \
0         다음         2576100             3869700                 0.0   

   monthlyAveMobileClkCnt  monthlyAvePcCtr  monthlyAveMobileCtr  plAvgDepth  \
0                     0.0              0.0                  0.0           0   

  compIdx  
0      낮음  


In [30]:
list(df2.columns)

['relKeyword',
 'monthlyPcQcCnt',
 'monthlyMobileQcCnt',
 'monthlyAvePcClkCnt',
 'monthlyAveMobileClkCnt',
 'monthlyAvePcCtr',
 'monthlyAveMobileCtr',
 'plAvgDepth',
 'compIdx']

# 빈도수 사전 불러오기

In [1]:
import pandas as pd
import json

data = []

with open("word_count/word_count_25.json", 'r') as file:
    for line in file:
        json_obj = json.loads(line)
        data.append(json_obj)

df = pd.DataFrame(data).transpose()
df.columns = ['count']
df['count'] = pd.to_numeric(df['count'])  #데이터 타입을 숫자로 변환

# 언더바(_)를 기준으로 앞 단어는 word, 뒤 단어는 tag 컬럼으로 분리
df['word'] = df.index.map(lambda x: x.split('_')[0])
df['tag'] = df.index.map(lambda x: x.split('_')[1])

# 정리
df.reset_index(drop=True, inplace=True)
df = df[['word', 'tag', 'count']]
df

,word,tag,count
0,현대,NNP,1458
1,과학,NNG,14355
2,시대,NNG,35605
3,앞,NNG,170296
4,더욱,MAG,51777
...,...,...,...
1167562,황구렁이,NNP,1
1167563,석구렁이,NNP,1
1167564,흑질뱅이,NNP,1
1167565,호미부,NNG,1


In [2]:
sorted_df = df.sort_values(by='count', ascending=True)
sorted_df.reset_index(drop=True, inplace=True)

df_over100 = sorted_df[sorted_df["count"] > 100].reset_index(drop=True)
df_over100

,word,tag,count
0,페리오스,NNP,101
1,아츰,NNG,101
2,오도문,NNG,101
3,무라야마,NNP,101
4,장포,NNP,101
...,...,...,...
39183,사람,NNG,701971
39184,되,VV,899517
39185,없,VA,901000
39186,있,VA,1356829


In [62]:
df_keyword = pd.DataFrame(columns=['relKeyword',
 'monthlyPcQcCnt',
 'monthlyMobileQcCnt',
 'monthlyAvePcClkCnt',
 'monthlyAveMobileClkCnt',
 'monthlyAvePcCtr',
 'monthlyAveMobileCtr',
 'plAvgDepth',
 'compIdx'])

df_keyword

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,monthlyAvePcClkCnt,monthlyAveMobileClkCnt,monthlyAvePcCtr,monthlyAveMobileCtr,plAvgDepth,compIdx


In [35]:
from tqdm import tqdm

cnt = 0
for word in tqdm(df_over100["word"]):
    cnt += 1
    if cnt < 10:
        print(word)

  0%|          | 0/39188 [00:00<?, ?it/s]

100%|██████████| 39188/39188 [00:00<00:00, 1042497.34it/s]

페리오스
아츰
오도문
무라야마
장포
명조
플레어
다비
기화요초


In [50]:
df_over100["word"][2649:]

2649      ,”
2650      투약
2651     백무랑
2652     일정표
2653     장악되
        ... 
39183     사람
39184      되
39185      없
39186      있
39187      하
Name: word, Length: 36539, dtype: object

In [58]:
from tqdm import tqdm

# all_responses = []
for word in tqdm(df_over100["word"][2649:]):
    keyword = word

    params={}
    params['hintKeywords']=keyword
    params['showDetail']='1'

    response = requests.get(BASE_URL + uri, params=params, 
                            headers=get_header(method, uri, API_KEY, SECRET_KEY, CUSTOMER_ID))
    try:
        data = response.json()["keywordList"][0]
    except KeyError as exception:
        data = {'relKeyword': keyword,
                'monthlyPcQcCnt': 0,
                'monthlyMobileQcCnt': 0,
                'monthlyAvePcClkCnt': 0.0,
                'monthlyAveMobileClkCnt': 0.0,
                'monthlyAvePcCtr': 0.0,
                'monthlyAveMobileCtr': 0.0,
                'plAvgDepth': 0,
                'compIdx': "낮음"}

    all_responses.append(data)

100%|██████████| 36539/36539 [3:43:58<00:00,  2.72it/s]  


In [59]:
len(all_responses)

39188

In [64]:
all_responses[-1]

{'relKeyword': '하',
 'monthlyPcQcCnt': 1610,
 'monthlyMobileQcCnt': 17300,
 'monthlyAvePcClkCnt': 0.0,
 'monthlyAveMobileClkCnt': 0.0,
 'monthlyAvePcCtr': 0.0,
 'monthlyAveMobileCtr': 0.0,
 'plAvgDepth': 0,
 'compIdx': '낮음'}

In [66]:
df_keywords = pd.DataFrame(all_responses)
df_keywords.to_csv("search_keywords.csv", index=False)

In [1]:
import pandas as pd

df_keywords = pd.read_csv("search_keywords.csv")
df_keywords

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,monthlyAvePcClkCnt,monthlyAveMobileClkCnt,monthlyAvePcCtr,monthlyAveMobileCtr,plAvgDepth,compIdx
0,페리오스,< 10,< 10,0.0,0.0,0.00,0.00,0,낮음
1,아츰,< 10,70,0.0,0.0,0.00,0.00,0,낮음
2,오도문,< 10,130,0.0,0.0,0.00,0.00,0,낮음
3,무라야마,140,580,0.0,0.0,0.00,0.00,2,중간
4,장포,190,580,0.5,0.5,0.26,0.10,2,중간
...,...,...,...,...,...,...,...,...,...
39183,사람,4030,14100,1.0,0.7,0.03,0.01,3,높음
39184,되,960,5940,0.0,0.0,0.00,0.00,0,낮음
39185,없,20,230,0.0,0.0,0.00,0.00,0,낮음
39186,있,20,270,0.0,0.0,0.00,0.00,0,낮음


In [3]:
dummy_len = 1167567 - len(df_keywords)
dummy_len

1128379

In [4]:
dummy = [{'relKeyword': 'Dummy',
        'monthlyPcQcCnt': 0,
        'monthlyMobileQcCnt': 0,
        'monthlyAvePcClkCnt': 0.0,
        'monthlyAveMobileClkCnt': 0.0,
        'monthlyAvePcCtr': 0.0,
        'monthlyAveMobileCtr': 0.0,
        'plAvgDepth': 0,
        'compIdx': '낮음'} for i in range(dummy_len)]
len(dummy)


1128379

In [5]:
df_dummy = pd.DataFrame(dummy)
df_dummy

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,monthlyAvePcClkCnt,monthlyAveMobileClkCnt,monthlyAvePcCtr,monthlyAveMobileCtr,plAvgDepth,compIdx
0,Dummy,0,0,0.0,0.0,0.0,0.0,0,낮음
1,Dummy,0,0,0.0,0.0,0.0,0.0,0,낮음
2,Dummy,0,0,0.0,0.0,0.0,0.0,0,낮음
3,Dummy,0,0,0.0,0.0,0.0,0.0,0,낮음
4,Dummy,0,0,0.0,0.0,0.0,0.0,0,낮음
...,...,...,...,...,...,...,...,...,...
1128374,Dummy,0,0,0.0,0.0,0.0,0.0,0,낮음
1128375,Dummy,0,0,0.0,0.0,0.0,0.0,0,낮음
1128376,Dummy,0,0,0.0,0.0,0.0,0.0,0,낮음
1128377,Dummy,0,0,0.0,0.0,0.0,0.0,0,낮음


In [8]:
df_keywords = pd.concat([df_dummy, df_keywords], axis=0, ignore_index=True)
df_keywords

,relKeyword,monthlyPcQcCnt,monthlyMobileQcCnt,monthlyAvePcClkCnt,monthlyAveMobileClkCnt,monthlyAvePcCtr,monthlyAveMobileCtr,plAvgDepth,compIdx
0,Dummy,0,0,0.0,0.0,0.00,0.00,0,낮음
1,Dummy,0,0,0.0,0.0,0.00,0.00,0,낮음
2,Dummy,0,0,0.0,0.0,0.00,0.00,0,낮음
3,Dummy,0,0,0.0,0.0,0.00,0.00,0,낮음
4,Dummy,0,0,0.0,0.0,0.00,0.00,0,낮음
...,...,...,...,...,...,...,...,...,...
1167562,사람,4030,14100,1.0,0.7,0.03,0.01,3,높음
1167563,되,960,5940,0.0,0.0,0.00,0.00,0,낮음
1167564,없,20,230,0.0,0.0,0.00,0.00,0,낮음
1167565,있,20,270,0.0,0.0,0.00,0.00,0,낮음


In [9]:
df_keywords.to_csv("search_keywords.csv", index=False, encoding="utf-8")

In [13]:
df_keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1167567 entries, 0 to 1167566
Data columns (total 9 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   relKeyword              1167567 non-null  object 
 1   monthlyPcQcCnt          1167567 non-null  object 
 2   monthlyMobileQcCnt      1167567 non-null  object 
 3   monthlyAvePcClkCnt      1167567 non-null  float64
 4   monthlyAveMobileClkCnt  1167567 non-null  float64
 5   monthlyAvePcCtr         1167567 non-null  float64
 6   monthlyAveMobileCtr     1167567 non-null  float64
 7   plAvgDepth              1167567 non-null  int64  
 8   compIdx                 1167567 non-null  object 
dtypes: float64(4), int64(1), object(4)
memory usage: 80.2+ MB


In [12]:
df_keywords["monthlyPcQcCnt"] + df_keywords["monthlyMobileQcCnt"]

0                  0
1                  0
2                  0
3                  0
4                  0
             ...    
1167562    403014100
1167563      9605940
1167564        20230
1167565        20270
1167566    161017300
Length: 1167567, dtype: object

# 네이버 검색 API - 백과사전

In [4]:
# 네이버 검색 API - 백과사전 검색
import os
import sys
import urllib.request
client_id = "KYzYTDMZ5_pb8T2kv9Bf"
client_secret = "y5LWx0u11n"
encText = urllib.parse.quote("어휘")
url = "https://openapi.naver.com/v1/search/encyc.json?query=" + encText \
        + "&display=" + str(5) 

request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id", client_id)
request.add_header("X-Naver-Client-Secret", client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

{
	"lastBuildDate":"Fri, 03 May 2024 12:41:44 +0900",
	"total":4686,
	"start":1,
	"display":5,
	"items":[
		{
			"title":"<b>어휘<\/b>",
			"link":"https:\/\/terms.naver.com\/entry.naver?docId=696070&cid=60533&categoryId=60533",
			"description":"특정한 언어체계가 가지는 <b>어휘<\/b>소(語彙素) 또는 <b>어휘<\/b>항목(語彙項目)을 통칭하여 이른다. [<b>어휘<\/b>의 개념] 단어를 <b>어휘<\/b>라고 일컫는 일이 있으나 단어는 <b>어휘<\/b>를 구성하는 자료일 뿐이다. <b>어휘<\/b>가 풍부하다는 말은... ",
			"thumbnail":""
		},
		{
			"title":"<b>어휘<\/b>",
			"link":"https:\/\/terms.naver.com\/entry.naver?docId=580954&cid=46674&categoryId=46674",
			"description":" 특정한 언어체계가 가지는 <b>어휘<\/b>항목 또는 <b>어휘<\/b>소를 통칭하는 언어학용어. [<b>어휘<\/b>의 개념] 단어 하나하나를 <b>어휘<\/b>라고 일컫는 일이 있으나 단어는 <b>어휘<\/b>를 구성하는 자료일 뿐이다. <b>어휘<\/b>가 풍부하다는 말은... ",
			"thumbnail":""
		},
		{
			"title":"<b>어휘<\/b>",
			"link":"https:\/\/terms.naver.com\/entry.naver?docId=2274767&cid=42251&categoryId=51122",
			"description":"남북에서 두드러진 차이를 보이는 부분은 바로 ‘<b>어휘<\/b>’다. 된소리와 예사소리의 표기 차이, 방언에 따라 달라지는 <b>어휘<\/b>, 한자어가 다른 경우, 순화된 <b>